<a href="https://colab.research.google.com/github/plaban1981/Python_Code_Debugging_Assistant_using_crewai/blob/main/Automated_code_debugging_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Groq API

In [1]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

## Install dependencies

In [3]:
%pip install -qU crewai langchain langchain-groq crewai[tools]

## Create Custom Tool

In [4]:
import subprocess
from typing import Dict, Any
from crewai.tools import BaseTool

class CodeInterpreterTool(BaseTool):
    name: str = "Code Interpreter"
    description: str = "Executes Python code and returns the result or error"

    def _run(self, code: str) -> Dict[str, Any]:
        try:
            result = subprocess.run(
                ['python', '-c', code],
                capture_output=True,
                text=True,
                timeout=10
            )
            return {
                "success": result.returncode == 0,
                "output": result.stdout,
                "error": result.stderr
            }
        except Exception as e:
            return {"error": str(e)}

class AnalyzeCodeTool(BaseTool):
    name: str = "Analyze Code"
    description: str = "Analyzes Python code for syntax and logical errors"

    def _run(self, code: str) -> str:
        interpreter = CodeInterpreterTool()
        result = interpreter._run(code)

        if result.get("error"):
            return f"Error found: {result['error']}"
        elif not result.get("success"):
            return f"Code execution failed: {result.get('error', 'Unknown error')}"
        else:
            return f"Code executed successfully. Output: {result.get('output', 'No output')}"

class FixCodeTool(BaseTool):
    name: str = "Fix Code"
    description: str = "Attempts to fix identified errors in Python code"

    def _run(self, code: str, errors: str) -> str:
        # First try to fix basic syntax errors
        fixed_code = self._fix_syntax(code)

        # Test if the fixes worked
        interpreter = CodeInterpreterTool()
        result = interpreter._run(fixed_code)

        if result.get("success"):
            return f"""
            Fixed code:
            {fixed_code}

            Execution result:
            {result.get('output', 'No output')}
            """
        else:
            return f"""
            Attempted fix (but errors remain):
            {fixed_code}

            Remaining errors:
            {result.get('error', 'Unknown error')}
            """

    def _fix_syntax(self, code: str) -> str:
        # Basic syntax fixes
        lines = code.split('\n')
        fixed_lines = []

        for line in lines:
            # Fix missing colons after function definitions
            if line.strip().startswith('def') and not line.strip().endswith(':'):
                line = line.rstrip() + ':'

            # Fix indentation
            if line.strip().startswith(('def', 'class', 'if', 'for', 'while')):
                next_line_should_indent = True
            elif line.strip() and not line.startswith(' '):
                line = '    ' + line

            fixed_lines.append(line)

        return '\n'.join(fixed_lines)

## Create Agents Task and Crew

In [5]:
from crewai import Agent, Task, Crew, Process, LLM
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field


class correction(BaseModel):
    error: str = Field(..., description="List of original errors in the python code else None")
    fix: str = Field(..., description="Explanation of each fix applied for each error if any else None")
    rectifiled_code: str = Field(..., description="The corrected python code else the original code")

# Initialize the tools
analyze_tool = AnalyzeCodeTool()
fix_tool = FixCodeTool()
code_interpretor_tool = CodeInterpreterTool()

# Configure LLM with function calling
llm = LLM(
    #model="ollama/deepseek-r1:8b",
    model ='groq/llama-3.3-70b-versatile',
    temperature=0.0
)

# Create agents
manager_agent = Agent(
    role="Manager",
    goal="Oversee and coordinate the code analysis and correction process",
    backstory="I am an experienced software development manager who ensures code quality and coordinates between analysis and correction teams.",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

analyzer_agent = Agent(
    role="Code Analyzer",
    goal="Identify syntax and logical errors in Python code",
    backstory="I am an expert at analyzing Python code and identifying potential issues and bugs.",
    tools=[analyze_tool, code_interpretor_tool],
    verbose=True,
    llm=llm
)

corrector_agent = Agent(
    role="Code Corrector",
    goal="Fix identified errors in Python code and output in the required JSON format",
    backstory="""I am specialized in fixing Python code issues while maintaining code quality and best practices.
    I always provide my output in valid JSON format matching the required schema.""",
    tools=[fix_tool, code_interpretor_tool],
    verbose=True,
    llm=llm
)

# Create tasks
analysis_task = Task(
    description="""
    Analyze the provided Python code and identify any syntax or logical errors.
    CODE:
    {code}
    List each error found with a clear explanation of what's wrong.
    """,
    expected_output="A detailed list of identified errors in the code, including syntax errors and logical issues with explanations",
    agent=analyzer_agent
)

correction_task = Task(
    description="""
    Your task is to analyze and fix Python code based on the analyzer agent's findings.
    You must return your response in valid JSON format exactly matching this structure:
    {
        "error": "<error description or 'None'>",
        "fix": "<fix explanation or 'None'>",
        "rectifiled_code": "<corrected code>"
    }

    Important:
    1. The output MUST be valid JSON
    2. All fields are required strings
    3. Use "None" (as a string) if no errors or fixes
    4. Ensure the rectifiled_code contains the complete code
    5. Do not include any additional text or formatting outside the JSON object
    """,
    expected_output="""A JSON object with three fields:
    - error: String describing the errors found or "None"
    - fix: String explaining the fixes applied or "None"
    - rectifiled_code: String containing the complete corrected code""",
    agent=corrector_agent,
    output_pydantic=correction
)

# Create the crew with hierarchical process
debugging_crew = Crew(
    agents=[analyzer_agent, corrector_agent],
    tasks=[analysis_task, correction_task],
    process=Process.hierarchical,
    manager_agent=manager_agent
)



## Kickoff the Debug Crew

In [6]:
# Function to run the debugging process
def debug_code(code: str):
    result = debugging_crew.kickoff(
        inputs={
            "code": code
        }
    )
    # Run the debugging process
    print("Debugging code...\n",code)
    print("Debugging result:\n", result)
    print("----------------------------------------------------------------------")
    print("Debug Code CrewAI Pydantic Results \n")
    print("----------------------------------------------------------------------")
    print(result.pydantic)
    return result

In [7]:
code = """
def add_numbers(a):
  return a + b
"""
debug_code(code)

# Agent: Manager
## Task: 
    Analyze the provided Python code and identify any syntax or logical errors.
    CODE:
    
def add_numbers(a):
  return a + b

    List each error found with a clear explanation of what's wrong.
    


# Agent: Manager
## Thought: Thought: The task requires analyzing the provided Python code to identify any syntax or logical errors. The code is a simple function named `add_numbers` that takes one argument `a` and returns the sum of `a` and `b`. However, there is no definition of `b` within the function or as a global variable. This could potentially lead to a logical error. To identify all possible errors, I should first analyze the code using the Analyze Code tool.
## Using tool: Analyze Code
## Tool Input: 
"{\"code\": \"def add_numbers(a):\\n  return a + b\"}"
## Tool Output: 
Code executed successfully. Output: 


# Agent: Manager
## Thought: The code has a syntax error because the variable 'b' is not defined. 
Thought: Since the code has been analyze

CrewOutput(raw='{\n  "error": "The variable \'b\' is not defined in the function add_numbers(a)",\n  "fix": "Modified the function to take two arguments, \'a\' and \'b\', and return their sum",\n  "rectifiled_code": "def add_numbers(a, b):\\n    return a + b"\n}', pydantic=correction(error="The variable 'b' is not defined in the function add_numbers(a)", fix="Modified the function to take two arguments, 'a' and 'b', and return their sum", rectifiled_code='def add_numbers(a, b):\n    return a + b'), json_dict=None, tasks_output=[TaskOutput(description="\n    Analyze the provided Python code and identify any syntax or logical errors.\n    CODE:\n    \ndef add_numbers(a):\n  return a + b\n\n    List each error found with a clear explanation of what's wrong.\n    ", name=None, expected_output='A detailed list of identified errors in the code, including syntax errors and logical issues with explanations', summary='\n    Analyze the provided Python code and...', raw="1. Syntax error: The var